In [1]:
from dotenv import load_dotenv
import openai

In [3]:
load_dotenv()
client = openai.OpenAI()

In [15]:
question = input("사용자 요청 메시지")
response = client.chat.completions.create(
    model='gpt-4.1-mini',
    messages=[
        # AI 가 할일
        {"role":"system","content":"You are a helpful assistant.using locale language."},
        # 사용자 요청
        {"role":"user","content":question}
    ],
)
print(f'💬AI: {response.choices[0].message.content}')

💬AI: 죄송하지만 현재 저는 실시간으로 인터넷에 접속해 정보를 검색할 수 없습니다. 하지만 한국 표준시(KST)는 UTC+9 시간대에 있으므로, 사용자의 현재 시간에 맞춰 계산할 수 있습니다. 현재 컴퓨터나 휴대폰에서 시간을 확인하시는 것이 가장 정확합니다. 도움이 필요하시면 언제든 말씀해 주세요!


In [36]:
# 실시간 데이터는 LLM 의 학습 내용이 아닙니다.
# ㄴ 함수 호출 또는 tools 사용해서 답변을 할 수 있는 기능이 있습니다.

# 시간 구하기 함수
from datetime import datetime

def get_current_time():
    now = datetime.now().strftime('%H:%M:%S')
    print(f'log : {now}')
    return now

get_current_time()

# 날짜 구하기 함수 : 예정
def get_current_date():
    now = datetime.now().strftime('%Y:%m:%d')
    print(f'log : {now}')
    return now

get_current_date()


# gpt 가 사용할 함수 목록을 정의
myfunctions = [
    {
        'name' : 'get_current_time',
        'description':'현재 시간 출력. 포맷 HH:MM:SS',
    },
    {
        'name' : 'get_current_date',
        'description':'현재 시간 출력. 포맷 YYYY:MM:DD'
    }
]


log : 12:02:48
log : 2025:11:06


In [ ]:
question = '현재 시간 알려줘'
response = client.chat.completions.create(
    model='gpt-4.1-mini',
    messages=[
        {"role":"system","content":"You are a helpful assistant.using locale language."},
        {"role":"user","content":question}
    ],
    functions=myfunctions,
)

# response 는 ChatCompletion 타입 객체. 그안에 속성은 객체 타입 or 문자열
print(f'log response : {response.model_dump_json(indent=2)}')

log response : {
  "id": "chatcmpl-CYkffrFAsPqoeZ0vruIFFSPpHnzsO",
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": {
          "arguments": "{}",
          "name": "get_current_time"
        },
        "tool_calls": null
      }
    }
  ],
  "created": 1762398591,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_4c2851f862",
  "usage": {
    "completion_tokens": 11,
    "prompt_tokens": 81,
    "total_tokens": 92,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


In [ ]:
# 함수 호출을 하고 답변을 만드는 추가적인 요청이 필요합니다.
# 함수는 1개만 호출 가능함
import json
# fn_name = response.choices[0].message.function_call.name
fn_name = getattr(response.choices[0].message.function_call,'name',None)
print(f'🔄log fn_name: {fn_name}')
if fn_name:
    func_response = globals()[fn_name]() # 문자열로 된 함수를 호출하는 방법
    followup_response = client.chat.completions.create(
        model='gpt-4.1-mini',
        messages=[
            {"role":"system","content":"You are a helpful assistant.using locale language."},
            {"role":"user","content":f'{fn_name} 함수를 실행한 결과 {func_response} 이용하여 최종 응답 만들어줘.'}
        ],
        functions=myfunctions
    )
    print(followup_response.model_dump_json(indent=2))
    
    # 함수 실행한 두번째 요청 응답
    result = followup_response.choices[0].message.content
else:
    # 첫번째 요청 응답
    result = response.choices[0].message.content    

print(f'💬AI : {result}')

🔄log fn_name: get_current_time
log : 12:09:54
{
  "id": "chatcmpl-CYkfhm8nopWhcVtuMmcjNDUKvL0no",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "현재 시간은 12시 09분 54초입니다.",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1762398593,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_4c2851f862",
  "usage": {
    "completion_tokens": 15,
    "prompt_tokens": 100,
    "total_tokens": 115,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}
💬AI : 현재 시간은 12시 09분 54초입니다.
